## Guitar amplifier emulation research

In [ ]:
import numpy as np
import seaborn as sns
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import IPython
import git

Verify GPU support

In [ ]:
tf.config.list_physical_devices()

Defining required constants

In [ ]:
INPUT_PATH = "data/float32/test_in.wav"
OUTPUT_PATH = "data/float32/test_out.wav"

TEST_INPUT_PATH = "data/float32/test_in.wav"
TEST_OUTPUT_PATH = "data/float32/test_out.wav"

TRAIN_TIME = 160 # seconds
VAL_TIME = 20 # seconds

BATCH_SIZE = 2 ** 14
INFERENCE_BATCH_SIZE = 44

EPOCHS = 100

INPUT_SIZE = 150
MODEL_NAME = input()

if MODEL_NAME == "":
  raise ValueError("Enter model name")

Creating dataset from audio

In [ ]:
from dataset import RawWaveformLoader

train, valid = RawWaveformLoader.DataSplitter(input_path=INPUT_PATH, 
                                              output_path=OUTPUT_PATH, 
                                              train_time=TRAIN_TIME, 
                                              valid_time=VAL_TIME, 
                                              batch_size=BATCH_SIZE,
                                              input_size=INPUT_SIZE,
                                              preemphasis=True).split()



demo = RawWaveformLoader(input_path=TEST_INPUT_PATH, 
                         output_path=TEST_OUTPUT_PATH, 
                         batch_size=BATCH_SIZE,
                         input_size=INPUT_SIZE, 
                         duration=10, 
                         offset=86,
                         preemphasis=False)

In [ ]:
IPython.display.Audio(train.x, rate=44100)

In [ ]:
IPython.display.Audio(train.y, rate=44100)

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, Input

from metrics import error_to_signal

LEARNING_RATE = 0.001
HIDDEN_UNITS = 64

K.clear_session()

conv1d_strides = 12
conv1d_filters = 16
hidden_units = 36

model = Sequential()
model.add(Conv1D(conv1d_filters, 12, strides=conv1d_strides, activation=None, padding='same', input_shape=(INPUT_SIZE,1)))
model.add(Conv1D(conv1d_filters, 12, strides=conv1d_strides, activation=None, padding='same'))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation=None))

model.compile(optimizer=Adam(LEARNING_RATE), loss='mse', metrics=['mse', error_to_signal, 'mae'])

Let's define callbacks

In [ ]:
from keras.callbacks import BackupAndRestore, CSVLogger, TensorBoard

from callbacks import CheckpointAndSave, CosineAnnealingScheduler

from utils import save_attempt_data
    
FOLDER = "models"
subfolder = f"{FOLDER}/{MODEL_NAME}"

save_attempt_data(subfolder)

callbacks = [
  CosineAnnealingScheduler(T_max=4, eta_max=1e-2, eta_min=1e-4),
  BackupAndRestore(f'{subfolder}/backup'),
  CheckpointAndSave(f'{subfolder}/mse_checkpoint/', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  CheckpointAndSave(f'{subfolder}/ets_checkpoint/', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  CheckpointAndSave(f'{subfolder}/mae_checkpoint/', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
]

In [ ]:
model.fit(train, validation_data=valid, shuffle=True, epochs=EPOCHS, callbacks=callbacks)